In [1]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
import MetaTrader5 as mt5

from sesto.constants import CURRENCIES
import sesto.metatrader.data as mtd
from sesto.plot import plot_tradingview, plot_plotly
from sesto.indicators import SMA, EMA, RSI, ROC

MetaTrader 5 initialized successfully.


In [2]:
# GENERATE ALL PAIR COMBINATIONS
from itertools import permutations

print(f"Number of currencies: {len(CURRENCIES)}")
# Generate all unique pairs of currency symbols
pairs = list(permutations(CURRENCIES, 2))
print(f"Number of pairs: {len(pairs)}")

Number of currencies: 9
Number of pairs: 72


In [3]:
# concatenate pairs into a single list
PAIRS = [pair[0] + pair[1] for pair in pairs]

In [4]:
TIMEFRAME = mt5.TIMEFRAME_H1
BARS = 500
MA_PERIODS = [5, 10, 20]
CSM_PERIOD = 10

In [5]:
mtd.fill_data(PAIRS, TIMEFRAME, BARS)

Failed to fetch data for symbol: USDEUR
No data fetched for pair: USDEUR
Failed to fetch data for symbol: USDGBP
No data fetched for pair: USDGBP
Fetched data for pair: USDJPY
Fetched data for pair: USDCHF
Fetched data for pair: USDCAD
Failed to fetch data for symbol: USDAUD
No data fetched for pair: USDAUD
Fetched data for pair: USDCHF
Failed to fetch data for symbol: USDNZD
No data fetched for pair: USDNZD
Fetched data for pair: EURUSD
Fetched data for pair: EURGBP
Fetched data for pair: EURJPY
Fetched data for pair: EURCHF
Fetched data for pair: EURCAD
Fetched data for pair: EURAUD
Fetched data for pair: EURCHF
Fetched data for pair: EURNZD
Fetched data for pair: GBPUSD
Failed to fetch data for symbol: GBPEUR
No data fetched for pair: GBPEUR
Fetched data for pair: GBPJPY
Fetched data for pair: GBPCHF
Fetched data for pair: GBPCAD
Fetched data for pair: GBPAUD
Fetched data for pair: GBPCHF
Fetched data for pair: GBPNZD
Failed to fetch data for symbol: JPYUSD
No data fetched for pair:

In [6]:
pairs_to_delete = [pair for pair, data in mtd.data.items() if data is None or len(data) == 0]
for pair in pairs_to_delete:
    del mtd.data[pair]

# Get the length of valid pair keys in mtd.data that have data
valid_pairs = sum(1 for pair, data in mtd.data.items() if data is not None and len(data) > 0)
print(f"Number of valid pairs with data: {valid_pairs}")

Number of valid pairs with data: 28


In [7]:
for symbol, df in mtd.data.items():
    for period in MA_PERIODS:
        SMA(df, period)
        EMA(df, period)
        ROC(df, period)
        RSI(df, period)
    
    df['psm'] = df[f'rsi-{CSM_PERIOD}'] + df[f'roc-{CSM_PERIOD}'] / 2

In [8]:
csm_data = {}

for currency in CURRENCIES:
    # Initialize a list to hold DataFrames of PSM values for the current currency
    psm_dfs = []
    
    # Iterate over all pair data
    for pair, df in mtd.data.items():
        if df is None or df.empty:
            continue
        
        base_currency = pair[:3]
        quote_currency = pair[3:]
        
        # Determine if the current currency is the base or quote in the pair
        if currency == base_currency:
            # Use PSM as is for base currency
            psm_series = df[['time', 'psm']].copy()
        elif currency == quote_currency:
            # Invert PSM for quote currency
            psm_series = df[['time', 'psm']].copy()
            psm_series['psm'] = -psm_series['psm']
        else:
            # Current currency not involved in this pair
            continue
        
        # Rename the 'psm' column to the pair name
        psm_series.rename(columns={'psm': pair}, inplace=True)
        
        # Append to the list of PSM DataFrames
        psm_dfs.append(psm_series)
    
    if not psm_dfs:
        print(f"No pairs found for currency: {currency}")
        continue
    
    # Merge all PSM DataFrames on 'time' using outer join to preserve all timestamps
    csm_df = psm_dfs[0]
    for psm_df in psm_dfs[1:]:
        csm_df = pd.merge(csm_df, psm_df, on='time', how='outer')
    
    # Sort by time to maintain chronological order
    csm_df.sort_values('time', inplace=True)
    
    # Reset index after sorting
    csm_df.reset_index(drop=True, inplace=True)
    
    # Handle missing PSM values by excluding them from the mean calculation
    # (NaNs are automatically ignored by pandas' mean function with skipna=True)
    
    # Calculate the mean PSM value across all relevant pairs for each candle
    csm_df['csm'] = csm_df.drop('time', axis=1).mean(axis=1)

    min_csm = csm_df['csm'].min()
    max_csm = csm_df['csm'].max()
    csm_df['csm'] = 100 * (csm_df['csm'] - min_csm) / (max_csm - min_csm)
    
    # Store the CSM DataFrame in the dictionary
    csm_data[currency] = csm_df

In [9]:
for currency, df in csm_data.items():
    plot_plotly(df, currency, ['csm'])